# Load libraries

In [ ]:
### Load libraries 
import os
import numpy as np
import pandas as pd

#import sqlite3 as sl
import uuid
#import csv
#import os.path
#from os import path
import glob
#import time
import re
from datetime import datetime

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from ipywidgets import *
from IPython.display import clear_output, display
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import qgrid

import helpers.research_helpers as app

CSV_SEPARATOR = ','     # '\t'

TBL_SURVEYS = 'SURVEYS'
TBL_MEMBERS = 'MEMBERS'
TBL_MEMBERS_TO_SURVEYS = 'MEMBERS_TO_SURVEYS'

SURVEYS_COL_SURVEY_ID = 'survey_id'
SURVEYS_COL_NAME = 'name'
SURVEYS_COL_DATE = 'date'

MEMBERS_COL_SURVEY_ID = 'member_id'
MEMBERS_COL_FNAME = 'first_name'
MEMBERS_COL_LNAME = 'last_name'
MEMBERS_COL_EMAIL = 'email'
MEMBERS_COL_LAST_SURVEY_DATE = 'last_survey_date'

print('Notebook initialized.')

# Load DB and All Active Roster

In [ ]:
### Load All Active Roster and other DB files 

df_complete_list = app.load_csv(app.FLDR_SAMPLING + '/All Active Roster.csv')

df_complete_list = df_complete_list.rename(columns=str.lower)
df_complete_list = df_complete_list[['member_id','email', 'firstname', 'lastname', 'job_title']]
df_complete_list['email'] = df_complete_list['email'].str.lower()

df_complete_list['member_id'] = df_complete_list['member_id'].astype('string')
df_complete_list['email'] = df_complete_list['email'].astype('string')
df_complete_list['firstname'] = df_complete_list['firstname'].astype('string')
df_complete_list['lastname'] = df_complete_list['lastname'].astype('string')
df_complete_list['job_title'] = df_complete_list['job_title'].astype('string')
#df_complete_list['last_survey_date'] = df_complete_list['last_survey_date'].astype('category')

if os.path.exists(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv'):
    df_surveyed_members = pd.read_csv (app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv' , sep=CSV_SEPARATOR, keep_default_na=False)
    df_surveyed_members.columns= df_surveyed_members.columns.str.lower()
    #df_surveyed_members.head()
else:
    df_surveyed_members = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])
    
if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS.csv'):
    df_surveys = pd.read_csv (app.FLDR_SAMPLING + '/SURVEYS.csv' , sep=CSV_SEPARATOR, keep_default_na=False)
    df_surveys.columns= df_surveys.columns.str.lower()
    #df_surveys.head()
else:
    df_surveys = pd.DataFrame(columns = ['survey_id','name','date'])
    
if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv'):
    df_surveys_to_members = pd.read_csv (app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv' , sep=CSV_SEPARATOR, keep_default_na=False)
    df_surveys_to_members.columns= df_surveys_to_members.columns.str.lower()
    #df_surveys_to_members.head()
else:
    df_surveys_to_members = pd.DataFrame(columns = ['survey_id','member_id'])

print(str(len(df_complete_list)) + ' records loaded from ' + app.FLDR_SAMPLING + '/All Active Roster.csv')
print(str(len(df_surveyed_members)) + ' records loaded from ' + app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv')
print(str(len(df_surveys)) + ' records loaded from ' + app.FLDR_SAMPLING + '/SURVEYS.csv')
print(str(len(df_surveys_to_members)) + ' records loaded from ' + app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv')

# Determine which survey directories should be save to DB

In [ ]:
### Please select the survey directories you would like to use. 

def get_date_name (row):
    new_val = str(row['date']) + '___' + str(row['name'])
    return new_val

def get_unprocessed_survey_dirs(df_saved_surveys):
    # get list of processed survey dirs
    
    surveys_dict = {}
    print('Previously processed survey dirs: {}'.format(df_saved_surveys['name'].values))
    
    for index, row in df_saved_surveys.iterrows(): 
        surveys_dict[row[SURVEYS_COL_NAME]] = row[SURVEYS_COL_DATE]

    # find potential unprocessed survey dirs by iterating over folder names in s_drive and, 
    # comparing names to previously processed list
    df = pd.DataFrame(columns=['name','date'])
    
    for filename in glob.iglob(app.FLDR_SAMPLING + '/**/*.[cC][sS][vV]', recursive=True):
        if 'qualtrics' in filename.lower():
            
            match_date = re.search(r'\d{4}-\d{2}-\d{2}', filename.split('/')[-1])
            if match_date != None:
                survey_date = datetime.strptime(match_date.group(), '%Y-%m-%d').date()
            else:
                t = os.path.getmtime(filename)
                survey_date = datetime.fromtimestamp(t)
            
            survey_name = filename.split('/')[-2]
            
            # verify survey_name starts with a year AND hasn't already been processed
            if survey_name not in surveys_dict:
                surveys_dict[survey_name] = survey_date
                #print('{}, on {}'.format(survey_name, survey_date.strftime('%Y-%m-%d')))
                new_row = {'name':survey_name, 'date': survey_date.strftime('%Y-%m-%d')}
                df = df.append(new_row, ignore_index=True)

    if len(df) > 0: # found new records, append to db
        return df
    else:
        df = pd.DataFrame(columns = [SURVEYS_COL_NAME, SURVEYS_COL_DATE])
        return df

### Get a dataframe of surveys that have not been processed
df_unprocessed_survey_dirs = get_unprocessed_survey_dirs(df_surveys)
print('Count of unprocessed survey dirs: {}'.format(str(len(df_unprocessed_survey_dirs))))
if df_unprocessed_survey_dirs is None:
    print('There are no new survey directories to process :)')
else:
    # Allow user's to select which surveys, if any, they want to exclude
    qgrid_sheet_to_keep = app.keep_grid_show_filter(df_unprocessed_survey_dirs, 'name', None, app.KEEP_LIST_SURVEY_DIRS)

## Apply changes

In [ ]:
### Apply filtering 
#
# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list_of_survey_dirs, app.KEEP_LIST_SURVEY_DIRS = app.keep_grid_apply_filter(qgrid_sheet_to_keep, 'name', df_unprocessed_survey_dirs, True)


# Process survey information in selected directories

In [ ]:
# will most likely delete
#df_current_file = None 
#df_post_merge = None
#df_current_dir = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])
#df_surveyed_members_to_processed = None

In [ ]:
### Iterate over directories and save info to db  

df_post_merge = None
df_current_dir = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])
df_surveyed_members_to_processed = None

def __remove_trailing_zero(member_id):
    str_member_id = str(member_id)
    
    if str_member_id[-2:] == '.0':
        str_member_id = str_member_id.replace('.0', '')
    else:
        str_member_id = uuid.uuid4()
    
    return str_member_id

for index, row in df_filtered_list_of_survey_dirs.iterrows(): 
    df_concatenated = pd.DataFrame(columns = ['email', 'last_survey_date', 'member_id', 'firstname', 'lastname', 'job_title'])

    print('folder  name: ' + row['name'])
    df_current_file = None
    df_post_merge = None
    df_current_dir = None
    #df_current_dir = pd.DataFrame(columns = ['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date'])
    
    survey_id = len(df_surveys.index)
    df_surveys['new_entry'] = 0
    survey_dir = {}
    survey_dir = {'survey_id': survey_id, 'name': row['name'], 'date': row['date'], 'new_entry': 1, }
    df_surveys = df_surveys.append(survey_dir, ignore_index = True)
    
    for filename in glob.iglob(app.FLDR_SAMPLING + '/' + row['name'] + '/**/*.[cC][sS][vV]', recursive=True):
        if 'qualtrics' in filename.lower():
            print('  -- ' + filename)
            
            # read csv from directory
            #df_current_file = None
            df_current_file = pd.read_csv(filename)
            df_current_file = df_current_file.rename(columns=str.lower)
            df_current_file['last_survey_date'] = row['date']
            df_current_file['survey_name'] = row['name']
            df_current_file['survey_id'] = survey_id
            df_current_file['email'] = df_current_file['email'].str.lower()
            df_current_file = df_current_file.rename({'fname' : 'first_name'}, axis=1)
            df_current_file = df_current_file.rename({'firstname' : 'first_name'}, axis=1)
            df_current_file = df_current_file.rename({'lname' : 'last_name'}, axis=1)
            df_current_file = df_current_file.rename({'lastname' : 'last_name'}, axis=1)
            df_current_file = df_current_file.drop(['first_name', 'last_name'], axis=1)
            #df_current_file.head()
            #print('  -- survey respondents loaded from CSV: {}'.format(str(len(df_current_file))))
            #df_current_file.sort_values('email', inplace = True)
            df_current_file.drop_duplicates(subset ='email', keep = 'first', inplace = True)
            #print('  -- survey respondents (after dropping dups): {}'.format(str(len(df_current_file))))
            
            
            #merge current CSV with other CSVs processed this session
            df_surveyed_members_to_processed = pd.concat([df_surveyed_members_to_processed, df_current_file], axis=0)
            print('------df_surveyed_members_to_processed {}'.format(str(len(df_surveyed_members_to_processed))))
            
#df_surveyed_members_to_processed.head()
#print('df_surveyed_members_to_processed {}'.format(str(len(df_surveyed_members_to_processed))))

In [ ]:
#print('  ----- df_surveyed_members_to_processed: {}'.format(str(len(df_surveyed_members_to_processed))))
#df_my_temp = None
#df_my_temp = pd.merge(left=df_surveyed_members_to_processed, right=df_complete_list, how='left', left_on='email', right_on='email')
#print('  ----- df_my_temp: {}'.format(str(len(df_my_temp))))

# Save to db

In [ ]:
#merge with AllActive Roster to get names and member_ids
#print('  ----- df_surveyed_members_to_processed: {}'.format(str(len(df_surveyed_members_to_processed))))
df_post_merge = None
df_post_merge = pd.merge(left=df_surveyed_members_to_processed, right=df_complete_list, how='left', left_on='email', right_on='email')
df_post_merge.drop_duplicates(subset =['survey_id', 'email'], inplace = True)
#print('  -- df_post_merge: {}'.format(str(len(df_post_merge))))
df_post_merge['firstname'] = df_post_merge['firstname'].fillna('')
df_post_merge['lastname'] = df_post_merge['lastname'].fillna('')
df_post_merge['job_title'] = df_post_merge['job_title'].fillna('')
df_post_merge['member_id'] = df_post_merge.apply(lambda x: __remove_trailing_zero(str(x['member_id']).strip()), axis=1)     
print('  -- df_post_merge: {}'.format(str(len(df_post_merge))))
#df_post_merge.to_csv(app.FLDR_SAMPLING + '/TEMP.csv', mode = 'w', header = True, index=False, quotechar='"')


In [ ]:
#process df to determine surveys to members dataset
df_surveys_to_members_temp = None
df_surveys_to_members_temp = df_post_merge[['survey_id', 'member_id', 'email']]
df_surveys_to_members_temp.sort_values(['survey_id', 'email'], inplace = True)
df_surveys_to_members_temp.drop_duplicates(subset =['survey_id', 'email'], inplace = True)
df_surveys_to_members_temp = df_surveys_to_members_temp[['survey_id', 'member_id']]

df_surveys_to_members_temp = pd.concat([df_surveys_to_members_temp, df_surveys_to_members], axis=0)
df_surveys_to_members_temp.sort_values(['survey_id', 'member_id'], inplace = True)
df_surveys_to_members_temp.drop_duplicates(subset = ['survey_id', 'member_id'], keep = 'first', inplace = True)
df_surveys_to_members = df_surveys_to_members_temp

#if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv'):
#    df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'a', header = False, index=False, quotechar='"')
#else:
#    df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
#    #df_surveys_to_members_temp.head()

df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
print('  -- df_surveys_to_members_temp: {}'.format(str(len(df_surveys_to_members_temp))))

In [ ]:
#process df to determine surveys
df_surveys_temp = None
df_surveys_temp = df_post_merge[['survey_id', 'survey_name', 'last_survey_date']]
df_surveys_temp = df_surveys_temp.rename({'survey_name' : 'name'}, axis=1)
df_surveys_temp = df_surveys_temp.rename({'last_survey_date' : 'date'}, axis=1)
df_surveys_temp.sort_values(['date'], inplace = True)
df_surveys_temp.drop_duplicates(subset =['survey_id'], inplace = True)

df_surveys_temp = pd.concat([df_surveys_temp, df_surveys], axis=0)
df_surveys_temp.sort_values(['name', 'survey_id'], inplace = True)
df_surveys_temp.drop_duplicates(subset ='name', keep = 'first', inplace = True)
df_surveys = df_surveys_temp

#if os.path.exists(app.FLDR_SAMPLING + '/SURVEYS.csv'):
#    df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'a', header = False, index=False, quotechar='"')
#else:
#    df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'w', header = True, index=True, quotechar='"')
#    #df_surveys_temp.head()

df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'w', header = True, index=True, quotechar='"')    
print('  -- df_surveys_temp: {}'.format(str(len(df_surveys_temp))))

In [ ]:
#process df to determine surveyed members
df_surveyed_members_temp = None
df_surveyed_members_temp = df_post_merge[['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date']]
df_surveyed_members_temp.sort_values(['email','last_survey_date'], inplace = True)
df_surveyed_members_temp.drop_duplicates(subset =['email'], inplace = True)

df_surveyed_members_temp = pd.concat([df_surveyed_members_temp, df_surveyed_members], axis=0)
df_surveyed_members_temp.sort_values(['email','last_survey_date'], inplace = True)
df_surveyed_members_temp.drop_duplicates(subset =['email'], inplace = True)
df_surveyed_members = df_surveyed_members_temp

#if os.path.exists(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv'):
#    df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv', mode = 'a', header = Fa, index=False, quotechar='"')
#else:
#    df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
#    #df_surveyed_members_temp.head()

df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
print('  -- df_surveyed_members_temp: {}'.format(str(len(df_surveyed_members_temp))))

# Junk 

In [ ]:
#merge with AllActive Roster to get names and member_ids
print('  ----- df_surveyed_members_to_processed: {}'.format(str(len(df_surveyed_members_to_processed))))
df_post_merge = None
df_post_merge = pd.merge(left=df_surveyed_members_to_processed, right=df_complete_list, how='left', left_on='email', right_on='email')
print('  -- df_post_merge: {}'.format(str(len(df_post_merge))))
df_post_merge['firstname'] = df_post_merge['firstname'].fillna('')
df_post_merge['lastname'] = df_post_merge['lastname'].fillna('')
df_post_merge['job_title'] = df_post_merge['job_title'].fillna('')
df_post_merge['member_id'] = df_post_merge.apply(lambda x: __remove_trailing_zero(str(x['member_id']).strip()), axis=1)     
print('  -- df_post_merge: {}'.format(str(len(df_post_merge))))

#f_post_merge.sort_values('survey_id', inplace = True)
#df_post_merge.drop_duplicates(subset ='email', keep = 'first', inplace = True)

#process df to determine surveys to members dataset
df_surveys_to_members_temp = df_post_merge[['survey_id', 'member_id', 'email']]
df_surveys_to_members_temp.sort_values(['survey_id', 'email'], inplace = True)
df_surveys_to_members_temp.drop_duplicates(subset =['survey_id', 'email'], inplace = True)
df_surveys_to_members_temp = df_surveys_to_members_temp[['survey_id', 'member_id']]
#df_surveys_to_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TO_MEMBERS.csv', mode = 'w', header = True, index=False, quotechar='"')
df_surveys_to_members_temp.head()
print('  -- df_surveys_to_members_temp: {}'.format(str(len(df_surveys_to_members_temp))))

#process df to determine surveys
df_surveys_temp = df_post_merge[['survey_id', 'survey_name', 'last_survey_date']]
df_surveys_temp = df_surveys_temp.rename({'survey_name' : 'name'}, axis=1)
df_surveys_temp = df_surveys_temp.rename({'last_survey_date' : 'date'}, axis=1)
df_surveys_temp.sort_values(['date'], inplace = True)
df_surveys_temp.drop_duplicates(subset =['survey_id'], inplace = True)
#df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'a', header = False, index=False, quotechar='"')
df_surveys_temp.head()
print('  -- df_surveys_temp: {}'.format(str(len(df_surveys_temp))))

#process df to determine surveyed members
df_surveyed_members_temp = df_post_merge[['member_id','email', 'firstname', 'lastname', 'job_title', 'last_survey_date']]
df_surveyed_members_temp.sort_values(['email','last_survey_date'], inplace = True)
#df_surveyed_members_temp.drop_duplicates(subset =['email'], inplace = True)
#df_surveyed_members_temp = df_surveyed_members_temp[['survey_id', 'member_id']]
#df_surveyed_members_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYED_MEMBERS_TEMP.csv', mode = 'w', header = True, index=False, quotechar='"')
df_surveyed_members_temp.head()
print('  -- df_surveyed_members_temp: {}'.format(str(len(df_surveyed_members_temp))))


#merge current CSV with other CSVs processed this session
#df_surveyed_members_to_processed = pd.concat([df_surveyed_members_to_processed, df_current_file], axis=0)
#print('------df_surveyed_members_to_processed {}'.format(str(len(df_surveyed_members_to_processed))))



#print('  -- survey respondents (after determining member_id): {}'.format(str(len(df_post_merge))))
#df_post_merge.to_csv(app.FLDR_SAMPLING + '/SURVEYS_TEMP.csv', mode = 'w', header = True, index=False, quotechar='"')

#df_post_merge.drop_duplicates(subset ='email', keep = 'first', inplace = True)
#print('  -- survey respondents (after determining member_id): {}'.format(str(len(df_post_merge))))
            

In [ ]:
if len(df_current_dir) > 0:
    # save to /SURVEYS.csv
    df_surveys_temp = df_surveys[df_surveys['new_entry']==1]
    df_surveys_temp = df_surveys_temp.drop(['new_entry'], axis=1)
    
    #df_surveys_temp.to_csv(app.FLDR_SAMPLING + '/SURVEYS.csv', mode = 'a', header = False, index=False, quotechar='"')
    print('Saved {} to /SURVEYS.csv'.format(str(len(df_surveys_temp))))
    df_surveys['new_entry'] = 0

In [ ]:
print('Saved {} to df_current_dir'.format(str(len(df_current_dir))))
print('Saved {} to df_members'.format(str(len(df_members))))


print('Saved {} to df_surveys_to_members'.format(str(len(df_surveys_to_members))))
df_surveys_to_members.head()

print('Saved {} to df_surveyed_members'.format(str(len(df_surveyed_members))))
df_surveyed_members.head()